# Thermochemistry

In [10]:
# this line makes figures interactive in Jupyter notebooks
%matplotlib inline
from matplotlib import pyplot as plt

import numpy as np
from scipy.optimize import root_scalar
from scipy import constants

from pint import UnitRegistry
ureg = UnitRegistry()
Q_ = ureg.Quantity

# for convenience:
def to_si(quant):
    '''Converts a Pint Quantity to magnitude at base SI units.
    '''
    return quant.to_base_units().magnitude

In [11]:
# these lines are only for helping improve the display
import matplotlib_inline.backend_inline
matplotlib_inline.backend_inline.set_matplotlib_formats('pdf', 'png')
plt.rcParams['figure.dpi']= 300
plt.rcParams['savefig.dpi'] = 300
plt.rcParams['mathtext.fontset'] = 'cm'

In [12]:
# define some constants
Ru = Q_(constants.R, 'J/(K*mol)')
g0 = Q_(constants.g, 'm/s^2')

The thermochemical performance of a rocket is primarily represented using the characteristic velocity:

$$
c^* = \frac{p_c A_t}{\dot{m}} = \sqrt{\frac{\mathcal{R}_u T_c}{\textrm{MW} \, \gamma} \left( \frac{\gamma + 1}{2} \right)^{\frac{\gamma+1}{\gamma-1}} } \;,
$$ (eq_cstar)

which depends on the (combustion) chamber temperature $T_c$, gas molecular weight $\textrm{MW}$, and gas specific heat ratio $\gamma$.
The thrust coefficient $C_F$ mainly represents the performance of the nozzle, but it also depends on the specific heat ratio $\gamma$.

Up to this point, we have been provided or assumed these values, but they are actually a function of

* propellant(s)
* ratio of oxidizer to fuel

Given a propellant or combination of propellants and heating (due to combustion, nuclear reactions, or electricity),
the gas in the chamber and moving into the nozzle will form a mixture of chemical species at the state of chemical equilibrium. At this state, the forward and reverse rates of all chemical reactions are balanced, and the species remain at a fixed composition (as long as the temperature and/or pressure remain constant).

The amounts of the chemical species at the equilibrium are unknown, but can be determined based on the initial conditions using methods based on [reaction equilibrium constants](https://kyleniemeyer.github.io/computational-thermo/content/chemical-equilibrium/equilibrium-constant.html) or the [minimization of Gibbs free energy](https://kyleniemeyer.github.io/computational-thermo/content/chemical-equilibrium/equilibrium-alternative-methods.html).
If the temperature is not known/fixed, then it is also an unknown and must be found.

Fortunately, we can use software tools such as NASA's [CEA](https://cearun.grc.nasa.gov) or [Cantera](https://cantera.org) to find the equilibrium state for us. We'll focus on CEA here.

In [29]:
def get_area_ratio(pressure_ratio, gamma):
    '''Calculates area ratio based on specific heat ratio and pressure ratio.
    pressure ratio: chamber / exit
    area ratio: exit / throat
    '''
    return (
        np.power(2 / (gamma + 1), 1/(gamma-1)) * 
        np.power(pressure_ratio, 1 / gamma) *
        np.sqrt((gamma - 1) / (gamma + 1) /
                (1 - np.power(pressure_ratio, (1 - gamma)/gamma))
                )
        )

def root_area_ratio(pressure_ratio, area_ratio, gamma):
    ''' Returns zero for a given area ratio, pressure ratio, and gamma.
    pressure ratio: chamber / exit
    area ratio: exit / throat
    '''
    return area_ratio - get_area_ratio(pressure_ratio, gamma)

def get_thrust_coeff(pressure_ratio, gamma):
    '''Calculates thrust coefficient for optimum expansion.
    pressure ratio: chamber / exit
    area ratio: exit / throat
    '''
    return np.sqrt(
        2 * np.power(gamma, 2) / (gamma - 1) * 
        np.power(2 / (gamma + 1), (gamma + 1)/(gamma - 1)) * 
        (1 - np.power(1.0 / pressure_ratio, (gamma - 1)/gamma))
        )

def get_cstar(Tc, MW, gamma):
    '''Calculates cstar using chamber properties.'''
    return (
        np.sqrt(Ru * Tc / (gamma * MW)) * 
        (2 / (gamma + 1))**(-0.5*(gamma + 1)/(gamma - 1))
        )

## Fixed temperature and pressure

Let's first consider the problem where the pressure and temperature of the combustion/heating chamber are fixed and known, and determine the equilibrium composition of chemical species.
This problem is relevant to an isothermal process, or where temperature is a design variable, such
as in nuclear thermal or electrothermal rockets.

For example, say we have an arcjet operating on gaseous hydrazine (N<sub>2</sub>H<sub>4</sub>) 
as a propellant, with a chamber temperature of 5000 K and pressure of 50 psia. 
The nozzle area ratio is 100, and the arcjet will operate in vacuum.
For this system, determine the equilibrium composition, the average molecular weight, ratio of specific heats $\gamma$,
and then use these to get $c^*$, $C_F$, and $I_{\textrm{sp}}$.

In [CEA](https://cearun.grc.nasa.gov/), this is a `tp` problem, or fixed temperature and pressure problem.
We should expect that, at such high temperatures, the equilibrium state will have mostly one- and two-atom
molecules, based on the elements present: N<sub>2</sub>, H<sub>2</sub>, H, N, and HN.

The CEA plaintext input file looks like:

```text
prob tp
 
p,psia= 50  t,k= 5000

reac
name N2H4 mol 1.0

output siunits
end
```

and the output is (with the repeated input removed):

```text
*******************************************************************************

         NASA-GLENN CHEMICAL EQUILIBRIUM PROGRAM CEA2, FEBRUARY 5, 2004
                   BY  BONNIE MCBRIDE AND SANFORD GORDON
      REFS: NASA RP-1311, PART I, 1994 AND NASA RP-1311, PART II, 1996

 *******************************************************************************

               THERMODYNAMIC EQUILIBRIUM PROPERTIES AT ASSIGNED

                           TEMPERATURE AND PRESSURE

             REACTANT                    WT FRACTION      ENERGY      TEMP
                                          (SEE NOTE)     KJ/KG-MOL      K  
 NAME        N2H4                         1.0000000         0.000      0.000

 O/F=    0.00000  %FUEL=  0.000000  R,EQ.RATIO= 0.000000  PHI,EQ.RATIO= 0.000000

 THERMODYNAMIC PROPERTIES

 P, BAR            3.4474
 T, K             5000.00
 RHO, KG/CU M    5.5368-2
 H, KJ/KG         42058.0
 U, KJ/KG         35831.8
 G, KJ/KG       -103744.4
 S, KJ/(KG)(K)    29.1605

 M, (1/n)           6.677
 (dLV/dLP)t      -1.04028
 (dLV/dLT)p        1.4750
 Cp, KJ/(KG)(K)   11.1350
 GAMMAs            1.2548
 SON VEL,M/SEC     2795.1

 MOLE FRACTIONS

 *H               0.74177
 *H2              0.04573
 *N               0.00806
 *NH              0.00021
 *N2              0.20422
```

So, CEA not only provides the equilibrium composition in terms of mole fraction ($X_i$), but 
also the mean molecular weight of the mixture $MW$; 
thermodynamic properties and derivatives density $\rho$, enthalpy $h$, entropy $s$,
$\left(\partial \log V / \partial \log P\right)_T$, $\left(\partial \log V / \partial \log T\right)_P$,
specific heat $C_p = \partial h / \partial T)_P$, the ratio of specific heats ($\gamma$), 
and the sonic velocity (i.e., speed of sound) $a$. Some of these quantities are not particularly interesting to us right now, but we can use these quantities to find our quantities of interest.

In [15]:
area_ratio = 100
Tc = Q_(5000, 'K')
Pc = Q_(50, 'psi')

# output from CEA
MW = Q_(6.677, 'kg/kmol')
gamma = 1.2548

First, we need to find the exit pressure of the nozzle based on the area ratio:

In [37]:
# initial guesses for Pc / Pe
root = root_scalar(root_area_ratio, x0=1000, x1=2000, args=(area_ratio, gamma))
Pc_Pe = root.root
Pe = Pc / Pc_Pe
print(f"Exit pressure = {Pe.to('psi'): .2e~P}")

cstar = get_cstar(Tc, MW, gamma)
print(f"Cstar = {cstar.to('m/s'): .1f~P}")

CF0 = get_thrust_coeff(Pc_Pe, gamma)
# ambient pressure is zero in vacuum
CF = CF0 + (1/Pc_Pe) * area_ratio
print(f"C_F = {CF: .3f}")

Exit pressure = 2.54e-02 psi
Cstar = 3786.6 m/s
C_F =  1.884


In [39]:
Isp = CF * cstar / g0
print(f"Isp = {Isp.to('s'): .1f~P}")

Isp = 727.4 s


## Adiabatic combustion

